In [15]:
import subprocess
for i in range(2480785, 2480794):
    subprocess.run(["scancel", str(i)])

scancel: error: Kill job error on job id 2480788: Access/permission denied


In [23]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_script.sh"

with open(filename) as f:
    file = f.read()


batch_sizes = [256]
lrs = [5e-5]
num_layerss = [12]
num_headss = [8]
epochss = [100]
warmups = [10]
validation_epochss = [5]
preembed_nodess = [True, False]
text_encoders = ['clip', 't5']


exps = []
for lr in lrs:
    for batch_size in batch_sizes:
        for num_layers in num_layerss:
            for num_heads in num_headss:
                for epochs in epochss:
                    for warmup in warmups:
                        for validation_epochs in validation_epochss:
                            for preembed_nodes in preembed_nodess:
                                for text_encoder in text_encoders:
                                    exp_name = f"coco-{batch_size}-{lr}-{epochs}-{num_layers}-{text_encoder}{'_preembed' if preembed_nodes else ''}"
                                    exps.append(exp_name)
                                    
                                    result_dir = f"results/{exp_name}"
                                    if not os.path.exists(result_dir):
                                        os.makedirs(result_dir)

                                    file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                                    file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                                    file = re.sub('--epochs .*', f"--epochs {epochs} " + r"\\" , file)
                                    file = re.sub('--validation_epochs .*', f"--validation_epochs {validation_epochs} " + r"\\" , file)
                                    file = re.sub('--warmup .*', f"--warmup {warmup} " + r"\\" , file)
                                    file = re.sub('--num_heads .*', f"--num_heads {num_heads} " + r"\\" , file)
                                    file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
                                    file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
                                    file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)

                                    file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                                    file = re.sub('--output .*', f"--output {result_dir}/train.log", file)
                                    file = re.sub('--error .*', f"--error {result_dir}/train.log", file)

                                    if preembed_nodes and "--preembed_nodes \\" not in file:
                                        file += "\n--preembed_nodes \\"
                                    elif preembed_nodes == False and "--preembed_nodes \\" in file:
                                        split = file.split("\n--preembed_nodes \\")
                                        file = "".join(split)

                                    print(file)

                                    with open(filename, 'w') as f:
                                        f.write(file)

                                    subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results/coco-256-5e-05-100-12-clip_preembed/train.log
#SBATCH --error results/coco-256-5e-05-100-12-clip_preembed/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 contrastive_train.py \
--vision_tokens_train coco_visual_tokens \
--text_tokens_train coco_text_tokens \
--vision_tokens_val coco_val_visual_tokens \
--text_tokens_val coco_val_text_tokens \
--batch_size 256 \
--epochs 100 \
--warmup 10 \
--validation_epochs 5 \
--checkpoint_epochs 20 \
--lr 5e-05 \
--num_heads 8 \
--num_layers 12 \
--text_encoder clip \
--exp_name coco-256-5e-05-100-12-clip_preembed \
--preembed_nodes \
Submitted batch job 2481234
#!/bin/bash
#SBATCH --job-name=2565e-05
#SBATCH --output results/coco-256-5e-05-100-12-t5_preembed/train.log
#SBATCH --error results/coco-256-5e-05-100-12-t5_preembed/train.log

In [21]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_base.sh"

with open(filename) as f:
    file = f.read()

batch_sizes = [256]
lrs = [1e-5, 5e-5]
epochss = [100]
warmups = [10]
validation_epochss = [5]
image_encoders = ['vit', 'clip']
text_encoders = ['t5', 'clip']


exps = []
for lr in lrs:
    for batch_size in batch_sizes:
        for epochs in epochss:
            for warmup in warmups:
                for image_encoder in image_encoders:
                    for text_encoder in text_encoders:
                        for validation_epochs in validation_epochss:
                            exp_name = f"coco-{batch_size}-{lr}-{epochs}-{image_encoder}-{text_encoder}_base"
                            exps.append(exp_name)
                            
                            result_dir = f"results/{exp_name}"
                            if not os.path.exists(result_dir):
                                os.makedirs(result_dir)

                            file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                            file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                            file = re.sub('--epochs .*', f"--epochs {epochs} " + r"\\" , file)
                            file = re.sub('--validation_epochs .*', f"--validation_epochs {validation_epochs} " + r"\\" , file)
                            file = re.sub('--warmup .*', f"--warmup {warmup} " + r"\\" , file)
                            file = re.sub('--image_encoder .*', f"--image_encoder {image_encoder} " + r"\\" , file)
                            file = re.sub('--text_encoder .*', f"--text_encoder {text_encoder} " + r"\\" , file)
                            file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)

                            file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                            file = re.sub('--output .*', f"--output {result_dir}/train.log", file)
                            file = re.sub('--error .*', f"--error {result_dir}/train.log", file)

                            print(file)

                            with open(filename, 'w') as f:
                                f.write(file)

                            subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=2561e-05
#SBATCH --output results/coco-256-1e-05-100-vit-t5_base/train.log
#SBATCH --error results/coco-256-1e-05-100-vit-t5_base/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:2
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=32G

python3 contrastive_train_baseline.py \
--text_tokens_train coco_text_tokens \
--text_tokens_val coco_val_text_tokens \
--batch_size 256 \
--epochs 100 \
--warmup 10 \
--validation_epochs 5 \
--checkpoint_epochs 20 \
--image_encoder vit \
--text_encoder t5 \
--lr 1e-05 \
--exp_name coco-256-1e-05-100-vit-t5_base \
Submitted batch job 2481048
#!/bin/bash
#SBATCH --job-name=2561e-05
#SBATCH --output results/coco-256-1e-05-100-vit-clip_base/train.log
#SBATCH --error results/coco-256-1e-05-100-vit-clip_base/train.log
#SBATCH --time=71:00:00
#SBATCH --gres=gpu:rtxa5000:2
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=sc

In [2]:
exps

['coco-256-1e-05-100_preembed',
 'coco-256-1e-05-100',
 'coco-256-5e-05-100_preembed',
 'coco-256-5e-05-100',
 'coco-256-1e-06-100_preembed',
 'coco-256-1e-06-100',
 'coco-256-5e-06-100_preembed',
 'coco-256-5e-06-100']

In [3]:
import re
import subprocess


file = ""
filename = f"run_aro.sh"

with open(filename) as f:
    file = f.read()

exps = ['coco-256-1e-05-100_preembed',
 'coco-256-1e-05-100',
 'coco-256-5e-05-100_preembed',
 'coco-256-5e-05-100',
 'coco-256-1e-06-100_preembed',
 'coco-256-1e-06-100',
 'coco-256-5e-06-100_preembed',
 'coco-256-5e-06-100']

datasets = ['aro_vgr', 'aro_vga', 'aro_coco_order']

for exp in exps:
    for dataset in datasets:
        result_dir = f'results/{exp}'

        file = re.sub('--dataset .*', f"--dataset {dataset} " + r"\\" , file)
        file = re.sub('--exp_name .*', f"--exp_name {exp} " + r"\\" , file)

        file = re.sub('--job-name=.*', f"--job-name={dataset}", file)
        file = re.sub('--output .*', f"--output {result_dir}/{dataset}.log", file)
        file = re.sub('--error .*', f"--error {result_dir}/{dataset}.log", file)

        
        preembed_nodes = '_preembed' in exp

        if preembed_nodes and "--preembed_nodes \\" not in file:
            file += "\n--preembed_nodes \\"
        elif preembed_nodes == False and "--preembed_nodes \\" in file:
            split = file.split("\n--preembed_nodes \\")
            file = "".join(split)

        print(file)

        with open(filename, 'w') as f:
            f.write(file)

        subprocess.run(["sbatch", filename])


#!/bin/bash
#SBATCH --job-name=aro_vgr
#SBATCH --output results/coco-256-1e-05-100_preembed/aro_vgr.log
#SBATCH --error results/coco-256-1e-05-100_preembed/aro_vgr.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 aro_eval.py \
--dataset aro_vgr \
--exp_name coco-256-1e-05-100_preembed \
--preembed_nodes \
Submitted batch job 2473381
#!/bin/bash
#SBATCH --job-name=aro_vga
#SBATCH --output results/coco-256-1e-05-100_preembed/aro_vga.log
#SBATCH --error results/coco-256-1e-05-100_preembed/aro_vga.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 aro_eval.py \
--dataset aro_vga \
--exp_name coco-256-1e-05-100_preembed \
--preembed_nodes \
Submitted batch job 2473382
#!/bin/bash
#SBATCH --job-name=aro_coco_

In [4]:
import re
import subprocess


file = ""
filename = f"run_winoground.sh"

with open(filename) as f:
    file = f.read()

exps = ['coco-256-1e-05-100_preembed',
 'coco-256-1e-05-100',
 'coco-256-5e-05-100_preembed',
 'coco-256-5e-05-100',
 'coco-256-1e-06-100_preembed',
 'coco-256-1e-06-100',
 'coco-256-5e-06-100_preembed',
 'coco-256-5e-06-100']

for exp in exps:
    result_dir = f'results/{exp}'

    file = re.sub('--exp_name .*', f"--exp_name {exp} " + r"\\" , file)

    file = re.sub('--job-name=.*', f"--job-name=winoground", file)
    file = re.sub('--output .*', f"--output {result_dir}/winoground.log", file)
    file = re.sub('--error .*', f"--error {result_dir}/winoground.log", file)

    
    preembed_nodes = '_preembed' in exp

    if preembed_nodes and "--preembed_nodes \\" not in file:
        file += "\n--preembed_nodes \\"
    elif preembed_nodes == False and "--preembed_nodes \\" in file:
        split = file.split("\n--preembed_nodes \\")
        file = "".join(split)

    print(file)

    with open(filename, 'w') as f:
        f.write(file)

    subprocess.run(["sbatch", filename])


#!/bin/bash
#SBATCH --job-name=winoground
#SBATCH --output results/coco-256-1e-05-100_preembed/winoground.log
#SBATCH --error results/coco-256-1e-05-100_preembed/winoground.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 winoground_eval.py \
--exp_name coco-256-1e-05-100_preembed \
--preembed_nodes \
Submitted batch job 2473405
#!/bin/bash
#SBATCH --job-name=winoground
#SBATCH --output results/coco-256-1e-05-100/winoground.log
#SBATCH --error results/coco-256-1e-05-100/winoground.log
#SBATCH --time=05:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=16G

python3 winoground_eval.py \
--exp_name coco-256-1e-05-100 \
Submitted batch job 2473406
#!/bin/bash
#SBATCH --job-name=winoground
#SBATCH --output results/coco-256-5e-05-100_preembed